In [1]:
!pip install pymupdf easyocr opencv-python pillow # easy ocr
!pip install pymupdf4llm
!pip install pyhwp beautifulsoup4
!pip install langchain_community langchain_google_genai langchain_openai faiss-cpu

# llamaindex
!pip install llama-index llama-index-vector-stores-faiss llama-index-embeddings-huggingface llama-index-embeddings-openai llama-index-llms-gemini llama-index-llms-openai

  Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl.metadata (10 kB)
Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.7.0
    Uninstalling google-ai-generativelanguage-0.7.0:
      Successfully uninstalled google-ai-generativelanguage-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.11 requires google-ai-generativelanguage<1,>=0.7, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [2]:
# 필수 빌드 도구 & 라이브러리 설치
!apt-get update
!apt-get install -y build-essential libgsf-1-dev libxml2-dev libglib2.0-dev libiconv-hook-dev

# 소스코드 clone
!git clone https://github.com/mete0r/pyhwp.git
%cd pyhwp

# 설치
!python setup.py install

%cd ..

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,267 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages

In [3]:
import io
import os
import re
import cv2
import time
import fitz  # PyMuPDF
import pyhwp
import gdown
import torch
import easyocr
import zipfile
import pickle
import easyocr
import subprocess
import pymupdf4llm
import pandas as pd
import numpy as np
from PIL import Image
from datetime import datetime
from bs4 import BeautifulSoup, NavigableString, Comment

from langchain.schema import Document

## 기본 Utils (basic_utils.py)

In [4]:
def load_secret(key_name: str):

    # 1) Kaggle 환경: os.environ 또는 파일에서 읽기
    if key_name in os.environ:
        return os.environ[key_name]

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        return user_secrets.get_secret(key_name)
    except ImportError:
        pass

    # 2) Colab 환경인지 확인
    try:
        import google.colab.userdata as userdata
        try:
            return userdata.get(key_name)
        except KeyError:
            pass # Key not in Colab userdata
    except ImportError:
        pass

    raise KeyError(f"Secret '{key_name}' not found in Colab userdata, os.environ, or file.")


def download(id, filename):
    if os.path.isfile(filename) and os.path.getsize(filename) > 0:
        print(f"[skip] {filename} 이미 존재합니다.")
    else:
        gdown.download(id=id, output=filename, quiet=False)
        print(f'[ok] {filename} 다운로드 완료.')

        if filename.lower().endswith('.zip'):
            try:
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(os.path.dirname(filename) or '.') # Extract to directory of the file, or current if no directory
                print(f'[ok] {filename} 압축풀기 완료.')
            except zipfile.BadZipFile:
                print(f"[warning] {filename}은 유효한 zip 파일이 아닙니다.")

    return filename


def load_csv(filename):
  """Loads a CSV file into a pandas DataFrame."""
  try:
    df = pd.read_csv(filename)
    print(f"[ok] {filename} 로드 완료.")
    return df
  except FileNotFoundError:
    print(f"[error] {filename} 파일을 찾을 수 없습니다.")
    return None
  except Exception as e:
    print(f"[error] {filename} 로드 중 오류 발생: {e}")
    return None


def save_pages(pages, filename):
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
        print(f"디렉토리 '{directory}'를 생성했습니다.")
    with open(filename, 'wb') as f:
        pickle.dump(pages, f)
    print(f"'{filename}' 파일에 pages를 저장했습니다.")


def load_pages(filename):
    if not os.path.exists(filename):
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다.")
        return None
    with open(filename, 'rb') as f:
        pages = pickle.load(f)
    print(f"'{filename}' 파일에서 pages를 로드했습니다.")
    return pages


def timer(func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"실행 시간: {elapsed_time:.4f}초")
    return result

def ext(original_filename, ext='pkl'):
  base_filename, _ = os.path.splitext(original_filename)
  return f"{base_filename}.{ext}"


def firstname(original_filename):
    filename_with_extension = os.path.basename(original_filename)
    base_filename, _ = os.path.splitext(filename_with_extension)
    return base_filename

## PDF/HWP 문서 Utils (document_utils.py)

In [5]:
import os
import subprocess
from bs4 import BeautifulSoup, NavigableString, Comment
import easyocr
import re
import urllib.request
from pathlib import Path
import shutil
from textwrap import dedent
from langchain_text_splitters import RecursiveCharacterTextSplitter
import logging


def load_pymupdf(pdf_path, filename, langs=['ko','en'], zoom=3.0, gpu=True):
    """
    PDF 페이지별 텍스트 레이어 + EasyOCR 결과 병합
    - langs: EasyOCR 언어 리스트
    - zoom: 해상도 배율 (3.0이면 약 216dpi)
    - gpu: GPU 사용 여부
    """
    # 캐시 로드
    output_path = f'outputs/{filename}'
    if os.path.exists(output_path):
        return load_pages(output_path)

    # EasyOCR Reader 초기화
    reader = easyocr.Reader(langs, gpu=gpu)

    doc = fitz.open(f'files/{pdf_path}')
    pages = []

    for page_num, page in enumerate(doc, start=1):
        # 1. 텍스트 레이어 추출
        text_layer = page.get_text()

        # 2. 고해상도 렌더링
        matrix = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=matrix, alpha=False)
        img = Image.open(io.BytesIO(pix.tobytes("png")))

        # 3. EasyOCR 실행
        cv_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        results = reader.readtext(cv_img, detail=0)  # detail=0 → 텍스트만 리스트로 반환
        ocr_text = "\n".join(results)

        # 4. 병합
        merged_text = (text_layer or "").strip() + "\n" + (ocr_text or "").strip()

        print(f"[페이지 {page_num}] 텍스트 길이: {len(text_layer)}, OCR 길이: {len(ocr_text)}")
        pages.append(Document(page_content=merged_text))
        display(img)
        print(ocr_text)
    doc.close()

    # 캐시 저장
    save_pages(pages, output_path)
    return pages


def load_pymupdf4llm_easyocr(pdf_filename, metadata = {}):
    IMAGE_PATH = 'images'
    pdf_path = f'files/{pdf_filename}'
    output_path = f'outputs/{ext(pdf_filename)}'
    if os.path.exists(output_path):
        return load_pages(output_path)

    os.makedirs(IMAGE_PATH, exist_ok=True)

    # EasyOCR 리더 객체 초기화 (GPU 사용 가능 시 활용)
    reader = easyocr.Reader(['ko', 'en'], gpu=torch.cuda.is_available())

    # 1. PyMuPDF4LLM을 사용해 마크다운 생성 및 이미지 추출
    # write_images=True 옵션으로 이미지 추출
    # image_path 옵션으로 이미지 저장 위치 지정
    # page_chunks=True 옵션으로 페이지별로 분리된 결과 생성
    extracted_chunks = pymupdf4llm.to_markdown(pdf_path, write_images=True, image_path=IMAGE_PATH, page_chunks=True)

    # final_markdown = ""
    pages = []
    # 2. EasyOCR로 이미지 내 텍스트 인식 후 마크다운에 추가
    for chunk in extracted_chunks:
        page_text = chunk["text"]

        page_number = chunk["metadata"]["page"] # Get the page index from metadata
        # print('페이지: ', page_number)

        # 마크다운 내 이미지 참조 찾기 (예: ![image_name](images/image_name.png))
        # ([^\]]+)는 이미지 이름 부분을 캡처
        image_ref_pattern = re.compile(r"!\[(.*?)\]\((.*?)\)")
        matches = image_ref_pattern.finditer(page_text)

        # 이미지 참조를 순서대로 처리
        for match in reversed(list(matches)):
            image_path = match.group(2)

            # EasyOCR로 이미지 내 텍스트 인식
            try:
                ocr_results = reader.readtext(image_path)
                ocr_caption = " ".join([text for _, text, _ in ocr_results])
            except Exception as e:
                print(f"OCR 처리 중 오류 발생: {e}")
                ocr_caption = "OCR 처리 실패"

            # 마크다운 텍스트 업데이트 (이미지 참조 다음에 OCR 결과 삽입)
            original_match_string = match.group(0)
            replacement_string = f"{original_match_string}\n\n**OCR 텍스트:** {ocr_caption}\n"
            page_text = page_text[:match.start()] + replacement_string + page_text[match.end():]
        pages.append(Document(page_content=page_text, metadata=metadata))
        # final_markdown += page_text + "\n\n"

    save_pages(pages, output_path)
    return pages #final_markdown


def hwp_to_html(hwp_path, temp_html_dir):
    """
    hwp5html을 사용해 HWP를 HTML로 변환합니다.
    변환 실패 시 hwp5txt를, 그것도 실패하면 hwplib(Java)를 폴백으로 사용합니다.
    """
    # hwp5 로그를 숨기기 위한 설정
    logging.getLogger('hwp5html').setLevel(logging.WARNING)

    # hwplib jar 파일을 저장할 디렉토리 및 버전
    JAVA_MAIN_CLASS_NAME = "HwpTextExtractor"
    JAR_DIR = Path("./temp_hwplib")
    HWPLIB_JAR_VERSION = "1.1.10"
    HWPLIB_JAR_NAME = f"hwplib-{HWPLIB_JAR_VERSION}.jar"
    HWPLIB_JAR_PATH = JAR_DIR / HWPLIB_JAR_NAME
    HWPLIB_DOWNLOAD_URL = f"https://repo1.maven.org/maven2/kr/dogfoot/hwplib/{HWPLIB_JAR_VERSION}/{HWPLIB_JAR_NAME}"

    path_hwp = Path(hwp_path)
    path_html = Path(temp_html_dir) / f"{path_hwp.stem}.html"
    os.makedirs(temp_html_dir, exist_ok=True)

    def setup_hwplib_java():
        """hwplib JAR 파일 다운로드 및 자바 소스 컴파일"""
        JAVA_MAIN_CLASS_FILE = JAR_DIR / f"{JAVA_MAIN_CLASS_NAME}.java"
        # 수정된 자바 소스 코드 (args 배열의 첫 번째 요소 사용 및 OnlyMainParagraph 옵션 적용)
        JAVA_MAIN_CLASS_CODE = dedent("""
            import kr.dogfoot.hwplib.object.HWPFile;
            import kr.dogfoot.hwplib.reader.HWPReader;
            import kr.dogfoot.hwplib.tool.textextractor.TextExtractor;
            import kr.dogfoot.hwplib.tool.textextractor.TextExtractMethod;
            import java.io.File;

            public class HwpTextExtractor {
                public static void main(String[] args) {
                    if (args.length < 1) {
                        System.err.println("Usage: java HwpTextExtractor <hwp_file_path>");
                        System.exit(1);
                    }
                    // 수정: args 배열의 첫 번째 요소 사용
                    String hwpFilePath = args[0];
                    try {
                        HWPFile hwpFile = HWPReader.fromFile(hwpFilePath);
                        if (hwpFile != null) {
                            // 수정: OnlyMainParagraph 옵션을 사용하여 텍스트 추출
                            String extractedText = TextExtractor.extract(hwpFile, TextExtractMethod.OnlyMainParagraph);
                            System.out.print(extractedText);
                        }
                    } catch (Exception e) {
                        e.printStackTrace();
                        System.exit(1);
                    }
                }
            }
        """).strip()
        os.makedirs(JAR_DIR, exist_ok=True)
        if not HWPLIB_JAR_PATH.exists():
            print(f"INFO: hwplib JAR 파일 (버전 {HWPLIB_JAR_VERSION}) 다운로드...")
            try:
                urllib.request.urlretrieve(HWPLIB_DOWNLOAD_URL, HWPLIB_JAR_PATH)
                print("INFO: hwplib JAR 파일 다운로드 완료.")
            except Exception as e:
                raise RuntimeError(f"hwplib JAR 파일 다운로드 실패: {e}")
        if not JAVA_MAIN_CLASS_FILE.exists():
            with open(JAVA_MAIN_CLASS_FILE, 'w', encoding='utf-8') as f:
                f.write(JAVA_MAIN_CLASS_CODE)
        class_output_dir = JAR_DIR
        if not (class_output_dir / f"{JAVA_MAIN_CLASS_NAME}.class").exists():
            print("INFO: 자바 클래스 컴파일...")
            try:
                subprocess.run(
                    ["javac", "-cp", os.fspath(HWPLIB_JAR_PATH), "-d", os.fspath(class_output_dir), os.fspath(JAVA_MAIN_CLASS_FILE)],
                    check=True,
                    text=True,
                    encoding="utf-8",
                    capture_output=True
                )
                print("INFO: 자바 클래스 컴파일 완료.")
            except subprocess.CalledProcessError as e:
                print(f"컴파일 실패: stderr: {e.stderr}")
                raise RuntimeError("자바 소스 컴파일 실패. 소스코드, JDK 설치, 그리고 PATH 설정을 확인하세요.")

    def extract_text_with_hwplib(hwp_path):
        """hwplib(Java)를 사용하여 텍스트 추출"""
        try:
            setup_hwplib_java()
            cmd = [
                "java",
                "-cp",
                f"{os.fspath(HWPLIB_JAR_PATH)}{os.pathsep}{os.fspath(JAR_DIR)}",
                JAVA_MAIN_CLASS_NAME,
                os.fspath(hwp_path)
            ]
            result = subprocess.run(cmd, check=True, text=True, encoding="utf-8", capture_output=True)
            return result.stdout
        except FileNotFoundError:
            raise RuntimeError("Java 실행 파일을 찾을 수 없습니다. Java가 설치되어 있고 PATH에 추가되었는지 확인하세요.")
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"hwplib 실행 실패 (오류 코드: {e.returncode}). stderr: {e.stderr}")

    try:
        # 1. hwp5html 시도
        subprocess.run(
            ["hwp5html", "--output", os.fspath(path_html), "--html", os.fspath(path_hwp)],
            check=True,
            text=True,
            encoding="utf-8",
            capture_output=True
        )
        print("hwp5html 변환 성공.")
        with open(path_html, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
        return soup.body or soup
    except subprocess.CalledProcessError as e:
        # 2. hwplib 폴백 시도
        print(f"hwp5html 변환 실패 (오류 코드: {e.returncode}). stderr: {e.stderr}")
        print(f"hwplib(Java)로 최종 대체합니다.")
        try:
            text = extract_text_with_hwplib(hwp_path)
            return BeautifulSoup(f"<body><p>{text}</p></body>", 'html.parser').body
        except Exception as e:
            print(f"hwplib 처리 중 예상치 못한 오류 발생: {e}")
            raise e


def load_hwp5html_easyocr(hwp_filename, metadata = {}, max_chars_per_page=1500, min_items_per_page=2):
    """
    하이브리드 페이지 분리 규칙:
    - 글자 수 누적이 max_chars_per_page를 넘으면 페이지 분리
    - 제목(h1/h2/h3) 등장 시 페이지 분리
    - 이미지 앞뒤로 페이지 분리
    - 표도 글자 수에 포함
    """
    def extract_table_md(table):
        rows = []
        for tr in table.find_all("tr"):
            cells = [c.get_text(strip=True) for c in tr.find_all(["td", "th"])]
            if cells:
                rows.append(cells)
        if not rows:
            return []
        md = []
        md.append("| " + " | ".join(rows[0]) + " |")
        md.append("| " + " | ".join(["---"] * len(rows[0])) + " |")
        for row in rows[1:]:
            md.append("| " + " | ".join(row) + " |")
        return md

    TEMP_HTML_DIR = "temp_hwp_html"
    hwp_path = f'files/{hwp_filename}'
    output_path = f'outputs/{ext(hwp_filename)}'
    if os.path.exists(output_path):
        return load_pages(output_path)

    # 1) HWP -> HTML
    body = hwp_to_html(hwp_path, TEMP_HTML_DIR)

    # 2) HTML 파싱 + 하이브리드 페이지 분리 + EasyOCR
    reader = easyocr.Reader(['ko', 'en'], gpu=False)
    md_lines = []
    page_num = 1
    char_count = 0
    items_in_page = 0

    def new_page():
        nonlocal page_num, char_count, items_in_page
        if items_in_page >= min_items_per_page:
            md_lines.append("")  # 페이지 끝 공백 줄
            page_num += 1
            md_lines.append(f"## Page {page_num}")
            char_count = 0
            items_in_page = 0

    md_lines.append(f"## Page {page_num}")

    for elem in body.descendants:
        if isinstance(elem, Comment) or (isinstance(elem, NavigableString) and not str(elem).strip()):
            continue
        if not getattr(elem, "name", None):
            continue

        tag = elem.name.lower()

        # 제목 등장 시 페이지 분리
        if tag in ["h1", "h2", "h3"]:
            new_page()
            text = elem.get_text(" ", strip=True)
            if text:
                md_lines.append(text)
                char_count += len(text)
                items_in_page += 1
            continue

        # 문단
        if tag in ["p", "li"]:
            text = elem.get_text(" ", strip=True)
            if text and not elem.find(["table", "img"]):
                if char_count > max_chars_per_page:
                    new_page()
                md_lines.append(text)
                char_count += len(text)
                items_in_page += 1

        # 표
        if tag == "table":
            tbl_md = extract_table_md(elem)
            if tbl_md:
                if char_count > max_chars_per_page:
                    new_page()
                md_lines.extend(tbl_md)
                char_count += sum(len("".join(r)) for r in tbl_md)
                items_in_page += 1

        # 이미지
        if tag == "img":
            src = elem.get("src")
            if src:
                img_path = os.path.join(TEMP_HTML_DIR, os.path.basename(src))
                if os.path.exists(img_path):
                    # 이미지 앞에서 페이지 분리
                    if char_count > max_chars_per_page:
                        new_page()
                    ocr_text = reader.readtext(img_path, detail=0)
                    md_lines.append(f"![{os.path.basename(img_path)}]({img_path})")
                    md_lines.append("\n".join(ocr_text) if ocr_text else "_(인식된 텍스트 없음)_")
                    char_count += 50  # 이미지도 글자 수로 환산
                    items_in_page += 1
                    # 이미지 뒤에서 페이지 분리
                    new_page()

    pages = []
    pages_raw = re.split(r'## Page \d+\s*', "\n".join(md_lines).strip())
    for page_text in pages_raw:
        if page_text.strip():
            pages.append(Document(page_content=page_text.strip(), metadata=metadata))
    save_pages(pages, output_path)
    return pages # "\n".join(md_lines).strip()


def load_documents(filename, metadata = {}):
    def recursive_documents(documents):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=150, separators=['\n\n', '\n', ' ', ''], length_function=len)
        documents = text_splitter.split_documents(documents)
        return documents

    _, ext = os.path.splitext(filename)
    if ext.lower() == '.pdf':
        documents = load_pymupdf4llm_easyocr(filename, metadata)
    elif ext.lower() == '.hwp':
        documents = load_hwp5html_easyocr(filename, metadata)
    else:
        print(f"'{filename}'는 지원하지 않는 파일 형식입니다.")

    return recursive_documents(documents)


## 필요자료 다운로드

In [6]:
download('1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb', 'output.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb
From (redirected): https://drive.google.com/uc?id=1t9TWN25lsshk_tIXyh3Gx-NzfNAWRdeb&confirm=t&uuid=6caa381d-be80-4f7d-9be9-306f2bf1f38f
To: /content/output.zip
100%|██████████| 212M/212M [00:02<00:00, 71.0MB/s]


[ok] output.zip 다운로드 완료.
[ok] output.zip 압축풀기 완료.


'output.zip'

## CSV 파일 확인하기

In [7]:
df = load_csv('data_list.csv')
df.head()

[ok] data_list.csv 로드 완료.


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...
3,20240430918,0.0,도시계획위원회 통합관리시스템 구축용역,150000000.0,인천광역시,2024-04-18 16:26:32,2024-05-02 10:00:00,2024-05-09 16:00:00,- 사업명: 도시계획위원회 통합관리시스템 구축 용역\n- 용역개요: 도시계획위원회와...,hwp,인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp,\r\n \r\n \r\n도시계획위원회 통합관리시스템 구축\r\n제 안 요 청...
4,20240430896,0.0,봉화군 재난통합관리시스템 고도화 사업(협상)(긴급),900000000.0,경상북도 봉화군,2024-04-18 16:33:28,2024-04-26 09:00:00,2024-04-30 17:00:00,- 사업명: 봉화군 재난통합관리시스템 고도화 사업\n- 사업개요: 공동수급(공동이행...,hwp,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp,\r\n \r\n \r\n제안요청서\r\n \r\n사 업 명\r\n봉화...


## PDF 샘플 로딩

In [8]:
pdf_df = df[df['파일형식'] == 'pdf']
pdf_filename = pdf_df.iloc[0]['파일명'] # 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
pages = load_pymupdf4llm_easyocr(pdf_filename) # 추후 pages를 반환하도록 변경해야 함.
markdown_output = '\n\n---\n\n'.join([p.page_content for p in pages])
with open(ext(pdf_filename, 'md'), "w", encoding="utf-8") as f:
    f.write(markdown_output)
print(f"PDF 처리 완료. {ext(pdf_filename, 'md')} 파일 생성.")

'outputs/고려대학교_차세대 포털·학사 정보시스템 구축사업.pkl' 파일에서 pages를 로드했습니다.
PDF 처리 완료. 고려대학교_차세대 포털·학사 정보시스템 구축사업.md 파일 생성.


## HWP 샘플 로딩

In [9]:
# 예시 사용
hwp_filename = df.iloc[0]['파일명'] # 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
print(f"원본 파일명: {hwp_filename}")
print(f".pkl 파일명: {ext(hwp_filename)}")

filename_to_find = '전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp'
found_row = df[df['파일명'] == filename_to_find]

if not found_row.empty:
    print(f"'{filename_to_find}' 파일을 찾았습니다.")
    display(found_row)
else:
    print(f"'{filename_to_find}' 파일을 찾을 수 없습니다.")


원본 파일명: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
.pkl 파일명: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.pkl
'전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp' 파일을 찾았습니다.


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
20,20240903688,0.0,JST 공유대학(원) xAPI기반 LRS시스템 구축,116000000.0,전북대학교,2024-09-04 15:01:14,2024-09-05 09:00:00,2024-09-19 12:00:00,- 사업개요: JST 공유대학(원)에서 xAPI 기반 LRS시스템을 구축하는 사업\...,hwp,전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp,\r\n \r\n \r\n \r\n JST 공유대학(원) xAPI기반 LR...


In [10]:
# ===== 실행 예시 =====
hwp_filename = "전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp"
documents = load_hwp5html_easyocr(hwp_filename)
markdown_output = '\n\n---\n\n'.join([p.page_content for p in documents])
with open(ext(hwp_filename, 'md'), "w", encoding="utf-8") as f:
    f.write(markdown_output)
print(f"PDF 처리 완료. {ext(hwp_filename, 'md')} 파일 생성.")
print(markdown_output[:1200])

'outputs/전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.pkl' 파일에서 pages를 로드했습니다.
PDF 처리 완료. 전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.md 파일 생성.
|  |  |
| --- | --- |
| 「JST공유대학(원)xAPI기반LRS시스템 구축」제 안 요 청 서 |
|  |  |
「 JST 공유대학(원) xAPI 기반 LRS 시스템 구축 」
제 안 요 청 서
2024. 9.
| 사 업 명 | JST공유대학(원)xAPI기반LRS시스템 구축 |
| --- | --- |
| 주관기관 | 전북지역혁신플랫폼 대학교육혁신본부 |
사 업 명
JST 공유대학(원) xAPI 기반 LRS 시스템 구축
주관기관
전북지역혁신플랫폼 대학교육혁신본부
| Ⅰ |  | 사업안내 |
| --- | --- | --- |
Ⅰ
사업안내
1. 사업개요
□ 사 업 명: JST 공유대학(원) xAPI 기반 LRS 시스템 구축
□ 주관기관: 전북지역혁신플랫폼 대학교육혁신본부
□ 사업기간: 계약체결일로부터 75일까지
□ 사업예산: 금116,000,000원(금일억일천육백만원, 부가세 포함)
□ 계약방법: 제한(총액)경쟁입찰(협상에 의한 계약)
2. 추진배경 및 사업목적
□ 추진배경
❍ (학습 데이터의 통합 관리) 기존의 LMS 나 다양한 학습 플랫폼에서 발생하는 데이터를 통합적으로 관리하고 분석할 필요성 증가
❍ (개인 맞춤형 학습 환경 구축) 학습자의 다양한 학습 데이터를 수집 및 분석하여 학습자의 패턴을 파악하여 학습자에게 맞춤형 학습 경로 제공
❍ (데이터 기반의 의사결정) JST 공유대학(원) 학습 데이터를 분석하여 교육 정책 수립에 필요한 정보를 제공하고, 교육 효과를 객관적으로 평가 할 수 있는 교육 데이터 기반의 정책 의사결정 지원 체계 요구
❍ 초개인화 LRS(Learning Record Store) 구축: 빅데이터에 기반한 학습자의 학습분석을 바탕으로, 개인 맞춤별 학습경로를 제공하여 학습 효과 극 대화 필요
□ 사업목적

## 싱글턴 백터스토어 생성

In [20]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

def create_context(df):
    VERSION = 'v1'
    EMB_NAME = 'intfloat/multilingual-e5-large-instruct'
    STORED_FOLDER = f'faiss-{VERSION}'
    os.environ["GOOGLE_API_KEY"] = load_secret("GOOGLE_API_KEY")
    os.environ["OPENAI_API_KEY"] = load_secret("OPENAI_API_KEY")
    embedding = HuggingFaceEmbeddings(model_name=EMB_NAME, multi_process=True) # GPU가 2개이기 때문임

    def save_vectorstore(documents):
        # 새로운 문서가 있을 경우, 기존 인덱스에 추가
        print(f"{len(documents)}개의 새로운 문서를 vectorstore에 추가합니다.")
        vectorstore.add_documents(documents=documents)
        vectorstore.save_local(STORED_FOLDER) # 변경된 인덱스 저장
        print(f"업데이트된 vectorstore를 '{STORED_FOLDER}' 폴더에 저장했습니다.")
        return vectorstore

    def create_documents(df):
        """
        DataFrame의 각 행을 순회하며 LangChain Document 객체를 생성합니다.
        '텍스트' 컬럼을 page_content로 사용하고, 나머지 컬럼을 메타데이터로 추가합니다.
        """
        documents = []

        # NaN 값을 빈 문자열로 대체하여 메타데이터에 문제가 없도록 처리
        df = df.fillna('')

        for index, row in df.iterrows():
            # page_content는 '텍스트' 컬럼의 내용으로 설정
            print(f"이건 {index + 1}번째 문서: {row['사업명']}")
            page_content = str(row['텍스트'])

            # 메타데이터 딕셔너리 생성
            metadata = {
                '공고 번호': row['공고 번호'],
                '사업명': row['사업명'],
                '사업 금액': row['사업 금액'],
                '발주 기관': row['발주 기관'],
                '공개 일자': row['공개 일자'],
                '입찰 참여 시작일': row['입찰 참여 시작일'],
                '입찰 참여 마감일': row['입찰 참여 마감일'],
                '사업 요약': row['사업 요약'],
                '파일명': row['파일명']
            }

            # Document 객체 생성
            docs = load_documents(row['파일명'], metadata=metadata)
            documents.extend(docs)

        return documents

    def print_pipe(x):
        print("--- 랭체인 중간 결과 ---")
        print(x)
        print("-------------------------")
        return x

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def load_llm_gemini():
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash", temperature=0.2, thinking_budget=0,  # reasoning 비활성화
            max_output_tokens=1000,  # HuggingFacePipeline의 max_new_tokens에 해당
        )
        return llm

    # OpenAI LLM 로드 함수 (새로 추가)
    def load_llm_openai():
        llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.2, max_tokens=1000)
        return llm

    def create_chain(retriever, llm):
        prompt = PromptTemplate(
            template=(
                "다음 문맥만을 근거로 질문에 답변하세요.\n"
                "- 반드시 한국어로 답변하세요.\n"
                "- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n"
                "- 금액/기한/요건은 원문과 일치하게 유지하세요.\n\n"
                "문맥:\n{context}\n\n"
                "질문:\n{question}\n\n"
                "답변:"
            ),
            input_variables=["context", "question"]
        )
        chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt | llm | StrOutputParser()
        )
        return chain

    def create_default_chain(retriever, llm):
        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "다음 문맥만을 근거로 질문에 답변하세요.\n"
             "- 반드시 한국어로 답변하세요.\n"
             "- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n"
             "- 금액/기한/요건은 원문과 일치하게 유지하세요."
            ),
            ("human", "문맥:\n{context}\n\n질문:\n{question}")
        ])

        chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt | RunnableLambda(print_pipe) | llm | RunnableLambda(print_pipe) | StrOutputParser()
        )
        return chain

    # 1. 기존 FAISS 인덱스 로드 또는 새로 생성
    if os.path.exists(STORED_FOLDER):
        vectorstore = FAISS.load_local(STORED_FOLDER, embedding, allow_dangerous_deserialization=True)
        print(f"'{STORED_FOLDER}' 폴더에서 vectorstore를 로드했습니다.")
    else:
        # 인덱스가 없을 경우, 더미 문서를 사용해 인덱스 초기 생성
        dummy_docs = [Document(page_content="초기 생성을 위한 더미 문서입니다.")]
        vectorstore = FAISS.from_documents(documents=dummy_docs, embedding=embedding)
        print(f"새로운 vectorstore를 더미 문서로 생성했습니다.")
        documents = create_documents(df)
        save_vectorstore(documents)

    retriever = vectorstore.as_retriever()
    llm = load_llm_openai() # load_llm_gemini()
    chain = create_default_chain(retriever, llm)
    return save_vectorstore, chain

df = load_csv('data_list.csv')
# hwp_filename = "전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp"
# documents = load_documents(hwp_filename)
save_vectorstore, chain = create_context(df)
# vectorstore = save_vectorstore(documents)

[ok] data_list.csv 로드 완료.
'faiss-v1' 폴더에서 vectorstore를 로드했습니다.


In [12]:
# !rm -rf 'outputs/한국농어촌공사_아세안+3 식량안보정보시스템(AFSIS) 3단계 협력(캄보디아.pkl'
# !rm -rf 'temp_hwp_html/한국농어촌공사_아세안+3 식량안보정보시스템(AFSIS) 3단계 협력(캄보디아.html'
# !rm -rf temp_hwp_html
# !rm -rf 'outputs/한국농어촌공사_아세안+3 식량안보정보시스템(AFSIS) 3단계 협력(캄보디아.pkl'
# !rm -rf 'outputs/[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 국회 방송시스템 구축 및 지역의회 연계 개선 PMC 용역.pkl'

## 싱글턴 실행하기

In [13]:
print(timer(lambda: chain.invoke("컨설팅 요구사항에 대해서 설명해 주세요.")))

--- 랭체인 중간 결과 ---
messages=[SystemMessage(content="다음 문맥만을 근거로 질문에 답변하세요.\n- 반드시 한국어로 답변하세요.\n- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n- 금액/기한/요건은 원문과 일치하게 유지하세요.", additional_kwargs={}, response_metadata={}), HumanMessage(content='문맥:\n| 산출정보 |  |\n| 관련 요구사항 |  |\n\n| 산출정보 |  |\n| 관련요구사항 |  |\n\n| 산출정보 |  |\n| 관련요구사항 |  |\n\n연 번\n구분\n설명\n개수\n1\n컨설팅 요구사항\n(CSR, Consulting Requirement)\n- 정보 자산의 기밀성과 무결성을 확보하기 위 해 목표 시스템의 데이터 및 기능, 운영 접근 을 통제하기 위한 요구사항을 기술\n8\n2\n데이터 요구사항\n(DAR, Data Requirement)\n- 정보시스템 구축을 위한 DB 구축 및 데이터 구조 설계 요구사항을 기술\n7\n3\n보안 요구사항\n(SER, Security Requirement)\n- 정보 자산의 기밀성과 무결성을 확보하기 위 해 목표 시스템의 데이터 및 기능, 운영 접근 을 통제하기 위한 요구사항을 기술\n5\n4\n품질 요구사항\n(QUR, Quality Requirement)\n- 목표 사업의 원활한 수행 및 운영을 위해 관리가 필요한 품질 항목, 품질 평가 대상 및 목표에 대 한 요구사항을 기술\n1\n5\n제약 요구사항\n(COR, Constraint Requirement)\n- 목표시스템 설계, 구축, 운영과 관련하여 사전 에 파악된 기술 · 표준 · 업무 · 법제도 등 제약조건 등을 파악하여 기술\n3\n6\n프로젝트 관리 요구사항\n(PMR, Project Management Requirement)\n- 프로젝트의 원활한 수행을 위한 관리 방법 및 추진 단계별 수행방안에 대한 요구사항을 기술\n5\n7\n프

In [14]:
print(timer(lambda: chain.invoke('전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축에 관련하여 설명해 주세요.')))

--- 랭체인 중간 결과 ---
messages=[SystemMessage(content="다음 문맥만을 근거로 질문에 답변하세요.\n- 반드시 한국어로 답변하세요.\n- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n- 금액/기한/요건은 원문과 일치하게 유지하세요.", additional_kwargs={}, response_metadata={}), HumanMessage(content='문맥:\n|  |  |\n| --- | --- |\n| 「JST공유대학(원)xAPI기반LRS시스템 구축」제 안 요 청 서 |\n|  |  |\n「 JST 공유대학(원) xAPI 기반 LRS 시스템 구축 」\n제 안 요 청 서\n2024. 9.\n| 사 업 명 | JST공유대학(원)xAPI기반LRS시스템 구축 |\n| --- | --- |\n| 주관기관 | 전북지역혁신플랫폼 대학교육혁신본부 |\n사 업 명\nJST 공유대학(원) xAPI 기반 LRS 시스템 구축\n주관기관\n전북지역혁신플랫폼 대학교육혁신본부\n| Ⅰ |  | 사업안내 |\n| --- | --- | --- |\nⅠ\n사업안내\n1. 사업개요\n□ 사 업 명: JST 공유대학(원) xAPI 기반 LRS 시스템 구축\n□ 주관기관: 전북지역혁신플랫폼 대학교육혁신본부\n□ 사업기간: 계약체결일로부터 75일까지\n□ 사업예산: 금116,000,000원(금일억일천육백만원, 부가세 포함)\n□ 계약방법: 제한(총액)경쟁입찰(협상에 의한 계약)\n2. 추진배경 및 사업목적\n□ 추진배경\n❍ (학습 데이터의 통합 관리) 기존의 LMS 나 다양한 학습 플랫폼에서 발생하는 데이터를 통합적으로 관리하고 분석할 필요성 증가\n❍ (개인 맞춤형 학습 환경 구축) 학습자의 다양한 학습 데이터를 수집 및 분석하여 학습자의 패턴을 파악하여 학습자에게 맞춤형 학습 경로 제공\n❍ (데이터 기반의 의사결정) JST 공유대학(원) 학습 데이터를 분석하여 교육 정책 수립에 필요한 정보를 제공하고, 교육 효과를 객관적으로 평가 할 수 있는 교

In [15]:
print(timer(lambda: chain.invoke('전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축에 연관하여, 요구사항 목록표 내용을 알려주세요.')))

--- 랭체인 중간 결과 ---
messages=[SystemMessage(content="다음 문맥만을 근거로 질문에 답변하세요.\n- 반드시 한국어로 답변하세요.\n- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n- 금액/기한/요건은 원문과 일치하게 유지하세요.", additional_kwargs={}, response_metadata={}), HumanMessage(content='문맥:\n|  |  |\n| --- | --- |\n| 「JST공유대학(원)xAPI기반LRS시스템 구축」제 안 요 청 서 |\n|  |  |\n「 JST 공유대학(원) xAPI 기반 LRS 시스템 구축 」\n제 안 요 청 서\n2024. 9.\n| 사 업 명 | JST공유대학(원)xAPI기반LRS시스템 구축 |\n| --- | --- |\n| 주관기관 | 전북지역혁신플랫폼 대학교육혁신본부 |\n사 업 명\nJST 공유대학(원) xAPI 기반 LRS 시스템 구축\n주관기관\n전북지역혁신플랫폼 대학교육혁신본부\n| Ⅰ |  | 사업안내 |\n| --- | --- | --- |\nⅠ\n사업안내\n1. 사업개요\n□ 사 업 명: JST 공유대학(원) xAPI 기반 LRS 시스템 구축\n□ 주관기관: 전북지역혁신플랫폼 대학교육혁신본부\n□ 사업기간: 계약체결일로부터 75일까지\n□ 사업예산: 금116,000,000원(금일억일천육백만원, 부가세 포함)\n□ 계약방법: 제한(총액)경쟁입찰(협상에 의한 계약)\n2. 추진배경 및 사업목적\n□ 추진배경\n❍ (학습 데이터의 통합 관리) 기존의 LMS 나 다양한 학습 플랫폼에서 발생하는 데이터를 통합적으로 관리하고 분석할 필요성 증가\n❍ (개인 맞춤형 학습 환경 구축) 학습자의 다양한 학습 데이터를 수집 및 분석하여 학습자의 패턴을 파악하여 학습자에게 맞춤형 학습 경로 제공\n❍ (데이터 기반의 의사결정) JST 공유대학(원) 학습 데이터를 분석하여 교육 정책 수립에 필요한 정보를 제공하고, 교육 효과를 객관적으로 평가 할 수 있는 교

## 멀티턴 벡터스토어 생성

In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory # Import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain # Import ConversationalRetrievalChain
from operator import itemgetter # Import itemgetter


def create_context(df):
    VERSION = 'v1'
    EMB_NAME = 'intfloat/multilingual-e5-large-instruct'
    STORED_FOLDER = f'faiss-{VERSION}'
    os.environ["GOOGLE_API_KEY"] = load_secret("GOOGLE_API_KEY")
    os.environ["OPENAI_API_KEY"] = load_secret("OPENAI_API_KEY")
    embedding = HuggingFaceEmbeddings(model_name=EMB_NAME, multi_process=True)

    def save_vectorstore(documents):
        # 새로운 문서가 있을 경우, 기존 인덱스에 추가
        print(f"{len(documents)}개의 새로운 문서를 vectorstore에 추가합니다.")
        vectorstore.add_documents(documents=documents)
        vectorstore.save_local(STORED_FOLDER) # 변경된 인덱스 저장
        print(f"업데이트된 vectorstore를 '{STORED_FOLDER}' 폴더에 저장했습니다.")
        return vectorstore

    def create_documents(df):
        """
        DataFrame의 각 행을 순회하며 LangChain Document 객체를 생성합니다.
        '텍스트' 컬럼을 page_content로 사용하고, 나머지 컬럼을 메타데이터로 추가합니다.
        """
        documents = []

        # NaN 값을 빈 문자열로 대체하여 메타데이터에 문제가 없도록 처리
        df = df.fillna('')

        for index, row in df.iterrows():
            # page_content는 '텍스트' 컬럼의 내용으로 설정
            print(f"이건 {index + 1}번째 문서: {row['사업명']}")
            metadata = {
                '공고 번호': row['공고 번호'],
                '사업명': row['사업명'],
                '사업 금액': row['사업 금액'],
                '발주 기관': row['발주 기관'],
                '공개 일자': row['공개 일자'],
                '입찰 참여 시작일': row['입찰 참여 시작일'],
                '입찰 참여 마감일': row['입찰 참여 마감일'],
                '사업 요약': row['사업 요약'],
                '파일명': row['파일명']
            }

            # Document 객체 생성
            docs = load_documents(row['파일명'], metadata=metadata)
            documents.extend(docs)

        return documents

    def print_pipe(x):
        print("--- 랭체인 중간 결과 ---")
        print(x)
        print("-------------------------")
        return x

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def load_llm_gemini():
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash", temperature=0.2, thinking_budget=0,  # reasoning 비활성화
            max_output_tokens=1000,  # HuggingFacePipeline의 max_new_tokens에 해당
        )
        return llm

    # OpenAI LLM 로드 함수 (새로 추가)
    def load_llm_openai():
        llm = ChatOpenAI(model="gpt-4o", temperature=0.2, max_tokens=1000)
        return llm

    # 기존 create_chain 함수 (유지)
    def create_chain(retriever, llm):
        prompt = PromptTemplate(
            template=(
                "다음 문맥만을 근거로 질문에 답변하세요.\n"
                "- 반드시 한국어로 답변하세요.\n"
                "- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n"
                "- 금액/기한/요건은 원문과 일치하게 유지하세요.\n\n"
                "문맥:\n{context}\n\n"
                "질문:\n{question}\n\n"
                "답변:"
            ),
            input_variables=["context", "question"]
        )
        chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt | llm | StrOutputParser()
        )
        return chain

    # 기존 create_chat_chain 함수 (수정: memory 파라미터를 optional로 변경)
    def create_chat_chain(memory=None): # Accept memory as an optional argument
        if memory is None:
            # 대화 기록을 위한 새로운 메모리 생성
            memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        # Conversational Retrieval Chain 생성
        chain = ConversationalRetrievalChain.from_llm(
            llm=llm, # Use llm from outer scope
            retriever=retriever, # Use retriever from outer scope
            memory=memory, # Use the provided or newly created memory
            # combine_docs_chain_kwargs={"prompt": your_custom_prompt} # 필요에 따라 커스텀 프rompt 사용
        )
        return chain

    # 새로운 runnable 형태의 대화형 체인 생성 함수 (유지)
    def create_runnable_chat_chain(retriever, llm):
        # 대화 기록을 위한 메모리 설정
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        # 사용자의 현재 질문과 대화 기록을 기반으로 검색할 쿼리를 생성하는 프롬프트
        question_generator_prompt = ChatPromptTemplate.from_messages([
            ("system", "주어진 대화 기록과 새로운 질문을 바탕으로 검색에 사용할 질문을 생성하세요."),
            ("human", "대화 기록:\n{chat_history}\n\n새로운 질문: {question}\n\n검색 질문:")
        ])

        # 검색된 문서와 사용자의 현재 질문을 바탕으로 답변을 생성하는 프롬프트
        answer_prompt = ChatPromptTemplate.from_messages([
            ("system",
             "다음 문맥만을 근거로 질문에 답변하세요.\n"
             "- 반드시 한국어로 답변하세요.\n"
             "- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n"
             "- 금액/기한/요건은 원문과 일치하게 유지하세요."
            ),
            ("human", "문맥:\n{context}\n\n질문:\n{question}")
        ])

        # 검색 질문 생성 체인
        question_generator_chain = question_generator_prompt | llm | StrOutputParser()

        # 답변 생성 체인
        document_chain = (
            {"context": itemgetter("context"), "question": itemgetter("question")}
            | answer_prompt | llm | StrOutputParser()
        )

        # 최종 대화형 검색 체인
        # 입력: {"question": "...", "chat_history": [...]}
        chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(lambda x: memory.load_memory_variables({})["chat_history"])
            ) |
            {
                "context": itemgetter("question") | question_generator_chain | retriever,
                "question": itemgetter("question")
            } |
            document_chain
        )

        # 메모리에 대화 기록을 저장하는 함수 (체인 실행 후 호출)
        def save_chat_history(inputs, outputs):
            memory.save_context(inputs, {"output": outputs})
            return outputs

        # 체인 실행 함수와 메모리 저장 함수를 함께 반환
        return chain, save_chat_history

    # 1. 기존 FAISS 인덱스 로드 또는 새로 생성
    if os.path.exists(STORED_FOLDER):
        vectorstore = FAISS.load_local(STORED_FOLDER, embedding, allow_dangerous_deserialization=True)
        print(f"'{STORED_FOLDER}' 폴더에서 vectorstore를 로드했습니다.")
    else:
        # 인덱스가 없을 경우, 더미 문서를 사용해 인덱스 초기 생성
        dummy_docs = [Document(page_content="초기 생성을 위한 더미 문서입니다.")]
        vectorstore = FAISS.from_documents(documents=dummy_docs, embedding=embedding)
        print(f"새로운 vectorstore를 더미 문서로 생성했습니다.")
        documents = create_documents(df)
        save_vectorstore(documents)

    retriever = vectorstore.as_retriever()
    llm = load_llm_openai() # load_llm_gemini()

    # Return save_vectorstore and create_chat_chain functions
    # create_chat_chain now accepts an optional memory parameter
    return save_vectorstore, create_chat_chain


df = load_csv('data_list.csv')
# Call create_context to get the necessary functions
save_vectorstore, create_chat_chain = create_context(df)

# Now you can create chat chains with separate memories as needed
# Example 1: Start a new conversation (memory is created internally)
# conversation1_chain = create_chat_chain()

# Example 2: Start another new conversation
# conversation2_chain = create_chat_chain()

# Example 3: Continue an existing conversation with a specific memory (advanced use case)
# existing_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# existing_conversation_chain = create_chat_chain(memory=existing_memory)

[ok] data_list.csv 로드 완료.
'faiss-v1' 폴더에서 vectorstore를 로드했습니다.


In [20]:
chat_chain = create_chat_chain()

In [21]:
# 대화 시작 예시
user_question = "전북대학교 JST 공유대학(원) 사업에 대해 설명해 주세요."
response = chat_chain.invoke({"question": user_question})

print("Assistant:", response['answer'])

Assistant: 전북대학교 JST 공유대학(원) 사업은 xAPI 기반의 LRS(학습 기록 저장소) 시스템 구축을 목표로 하고 있습니다. 이 사업은 전북지역혁신플랫폼 대학교육혁신본부가 주관하며, 학습 데이터의 통합 관리와 개인 맞춤형 학습 환경 구축을 목적으로 합니다. 이를 통해 학습자의 다양한 학습 데이터를 수집 및 분석하여 맞춤형 학습 경로를 제공하고, 데이터 기반의 의사결정을 지원하는 체계를 구축하려고 합니다. 사업 기간은 계약 체결일로부터 75일이며, 예산은 116,000,000원입니다. 이 시스템은 ADL의 xAPI 표준 체계를 지원하며, 학습자의 학습 활동 데이터를 실시간으로 저장 및 관리할 수 있는 기능을 갖추게 됩니다.


In [22]:
# 대화를 이어서 질문하는 예시
user_follow_up_question = "그 사업의 예산은 얼마인가요?"
response = chat_chain.invoke({"question": user_follow_up_question})

print("Assistant:", response['answer'])

Assistant: 전북대학교 JST 공유대학(원) xAPI 기반 LRS 시스템 구축 사업의 예산은 116,000,000원(금일억일천육백만원, 부가세 포함)입니다.


In [23]:
# 추가 질문 예시
user_additional_question = "그 사업 기간은 어떻게 되나요?"
response = chat_chain.invoke({"question": user_additional_question})

print("Assistant:", response['answer'])

Assistant: 전북대학교 JST 공유대학(원) xAPI 기반 LRS 시스템 구축 사업의 기간은 계약 체결일로부터 75일까지입니다.


## 작업파일 저장하기

In [16]:
import shutil
import os

def zip_folder(source_folder, output_filename):
    """
    지정된 폴더를 ZIP 파일로 압축합니다.

    Args:
        source_folder (str): 압축할 원본 폴더 경로.
        output_filename (str): 생성될 ZIP 파일의 이름 (확장자 제외).
    """
    # 압축 파일을 저장할 경로와 이름 지정
    # 예: 'archive_folder' 폴더를 압축하여 'archive.zip' 파일 생성
    # make_archive()는 확장자를 자동으로 추가합니다.
    shutil.make_archive(output_filename, 'zip', source_folder)
    print(f"'{source_folder}' 폴더가 '{output_filename}.zip'으로 압축되었습니다.")

# 사용 예시
source_dir = 'temp_hwp_html'  # 압축할 폴더 이름
output_name = 'hwp_html_archive'  # 생성될 ZIP 파일의 이름 (archive.zip)

# 만약 temp_hwp_html 폴더가 존재한다면
if os.path.exists(source_dir):
    zip_folder(source_dir, output_name)
else:
    print(f"'{source_dir}' 폴더가 존재하지 않습니다.")


# 사용 예시
source_dir = 'outputs'  # 압축할 폴더 이름
output_name = 'outputs_archive'  # 생성될 ZIP 파일의 이름 (archive.zip)

# 만약 temp_hwp_html 폴더가 존재한다면
if os.path.exists(source_dir):
    zip_folder(source_dir, output_name)
else:
    print(f"'{source_dir}' 폴더가 존재하지 않습니다.")

'temp_hwp_html' 폴더가 존재하지 않습니다.
'outputs' 폴더가 'outputs_archive.zip'으로 압축되었습니다.


## llama index 사용하기

In [17]:
import os
from pathlib import Path
from llama_index.core import VectorStoreIndex, Document, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts.prompts import PromptTemplate
import faiss

# 전역 설정
VERSION = 'v1'
EMB_NAME_HF = 'intfloat/multilingual-e5-large-instruct'
EMB_NAME_OAI = 'text-embedding-3-large'
STORED_FOLDER = f'faiss-{VERSION}'

# LangChain의 Document 객체를 LlamaIndex의 Document 객체로 변환하는 함수
def convert_lc_to_llamaindex_docs(lc_documents):
    llama_documents = []
    for doc in lc_documents:
        llama_doc = Document(text=doc.page_content, metadata=doc.metadata)
        llama_documents.append(llama_doc)
    return llama_documents

def create_context_llamaindex(df):

    # API 키 로드
    os.environ["GOOGLE_API_KEY"] = load_secret("GOOGLE_API_KEY")
    os.environ["OPENAI_API_KEY"] = load_secret("OPENAI_API_KEY")

    # 임베딩 모델 준비 (HuggingFace와 OpenAI)
    embedding_hf = HuggingFaceEmbedding(model_name=EMB_NAME_HF, multi_process=True)
    embedding_oai = OpenAIEmbedding(model=EMB_NAME_OAI)

    def create_documents(df):
        documents = []
        df = df.fillna('')
        for index, row in df.iterrows():
            print(f"이건 {index + 1}번째 문서: {row['사업명']}")
            metadata = {
                '공고 번호': row['공고 번호'],
                '사업명': row['사업명'],
                '사업 금액': row['사업 금액'],
                '발주 기관': row['발주 기관'],
                '공개 일자': row['공개 일자'],
                '입찰 참여 시작일': row['입찰 참여 시작일'],
                '입찰 참여 마감일': row['입찰 참여 마감일'],
                '사업 요약': row['사업 요약'],
                '파일명': row['파일명']
            }
            # 이 함수는 LangChain Document를 반환한다고 가정
            lc_docs = load_documents(row['파일명'], metadata=metadata)
            llama_docs = convert_lc_to_llamaindex_docs(lc_docs)
            documents.extend(llama_docs)
        return documents

    # 1. FAISS 인덱스 로드 또는 생성
    if os.path.exists(STORED_FOLDER):
        faiss_index = faiss.read_index(f"{STORED_FOLDER}/faiss_index.bin")
        faiss_vector_store = FaissVectorStore.from_faiss_index(faiss_index)
        storage_context = StorageContext.from_defaults(vector_store=faiss_vector_store)
        index = VectorStoreIndex.from_documents(documents=[], storage_context=storage_context, service_context=embedding_hf)
        print(f"'{STORED_FOLDER}' 폴더에서 vectorstore를 로드했습니다.")
    else:
        faiss_index = faiss.IndexFlatL2(embedding_hf.get_query_embedding_length())
        faiss_vector_store = FaissVectorStore(faiss_index=faiss_index)
        storage_context = StorageContext.from_defaults(vector_store=faiss_vector_store)
        index = VectorStoreIndex.from_documents(documents=[], storage_context=storage_context, service_context=embedding_hf)
        print(f"새로운 vectorstore를 생성했습니다.")

    # 2. 문서 생성 및 인덱스 업데이트
    documents = create_documents(df)
    index.insert_nodes(documents)

    # 3. LLM 설정 및 쿼리 엔진 생성
    llm_gemini = Gemini(temperature=0.2, model="gemini-2.5-flash")
    llm_openai = OpenAI(temperature=0.2, model="gpt-4o")

    def create_query_engine(llm, embedding_model):
        index.service_context.embed_model = embedding_model
        query_engine = index.as_query_engine(llm=llm, response_mode="compact")
        return query_engine

    query_engine_gemini = create_query_engine(llm_gemini, embedding_hf)
    query_engine_openai = create_query_engine(llm_openai, embedding_oai)

    return query_engine_gemini, query_engine_openai

# 사용 예시
df = load_csv('data_list.csv')
query_engine_gemini, query_engine_openai = create_context_llamaindex(df)

# 질문
response_gemini = query_engine_gemini.query("질문")
response_openai = query_engine_openai.query("질문")

print("Gemini 응답:", response_gemini)
print("OpenAI 응답:", response_openai)

[ok] data_list.csv 로드 완료.


TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'multi_process'

### 청킹작업시도 (Develop 필요)

In [ ]:
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List, Dict, Set
import numpy as np


def load_pymupdf(filename: str, chunk_size: int = 1000, chunk_overlap: int = 150, top_percentile: float = 15.0, min_important_text_length: int = 20, min_chunk_length_to_merge: int = 50) -> List[Document]:
    """
    폰트 크기, 최소 글자 수 필터링 및 문맥 기반 병합을 결합하여
    순서를 보장하며 PDF 문서를 청킹합니다.

    Args:
        filename (str): 처리할 PDF 파일의 이름.
        chunk_size (int): RecursiveCharacterTextSplitter의 청크 크기.
        chunk_overlap (int): RecursiveCharacterTextSplitter의 청크 오버랩.
        top_percentile (float): 상위 폰트 크기를 결정하는 퍼센타일 값 (예: 15.0).
        min_important_text_length (int): 중요 텍스트로 간주될 최소 길이.
        min_chunk_length_to_merge (int): 병합 대상이 될 최소 청크 길이.

    Returns:
        List[Document]: 청킹된 문서 리스트.
    """
    doc = fitz.open(f'{PDF_PATH}/{filename}')

    # 1단계: 폰트 크기 분포 및 반복 텍스트 분석 (이전과 동일)
    all_font_sizes = []
    first_page_spans: Dict[str, bool] = {}
    last_page_spans: Dict[str, bool] = {}

    for page_num, page in enumerate(doc):
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if block['type'] == 0:
                for line in block['lines']:
                    for span in line['spans']:
                        text = span['text'].strip()
                        if not text:
                            continue
                        all_font_sizes.append(round(span['size'], 2))

                        if page_num == 0:
                            first_page_spans[text] = True
                        if page_num == len(doc) - 1:
                            last_page_spans[text] = True

    if not all_font_sizes:
        doc.close()
        return []

    font_size_threshold = np.percentile(all_font_sizes, 100 - top_percentile)
    common_texts: Set[str] = set(first_page_spans.keys()) & set(last_page_spans.keys())

    # 2단계: 문서 순회하며 텍스트 덩어리 생성 및 병합 (순서 보장)
    combined_texts = []
    current_text = ""
    is_current_important = False

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if block['type'] == 0:
                for line in block['lines']:
                    for span in line['spans']:
                        text = span['text'].strip()
                        if not text:
                            continue

                        # 반복 문구 필터링
                        if text in common_texts:
                            continue

                        font_size = round(span['size'], 2)
                        is_important = font_size >= font_size_threshold

                        if not is_important and len(text) < 10:
                            continue

                        # 텍스트 유형이 바뀌는 지점을 청크 경계로 활용
                        # 단, 새로 시작하는 텍스트 덩어리가 짧으면 이전 덩어리에 병합
                        if current_text and is_important != is_current_important:
                            # 새로 시작하는 중요 텍스트가 짧으면 이전 덩어리에 병합
                            if is_important and len(text) < min_important_text_length:
                                current_text += " " + text
                                continue
                            # 이전 덩어리가 너무 짧으면 새로 분리하지 않고 계속 병합
                            elif len(current_text) < min_chunk_length_to_merge:
                                current_text += " " + text
                                is_current_important = is_important
                                continue
                            else:
                                combined_texts.append(current_text)
                                current_text = ""

                        current_text += text + " "
                        is_current_important = is_important

    if current_text:
        combined_texts.append(current_text)

    doc.close()
    print('큰 청킹 수: ', len(combined_texts))
    # 3단계: 순차적 청킹 적용 및 메타데이터 추가 (이전과 동일)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    processed_docs = []
    for text in combined_texts:
        cleaned_text = " ".join(text.split())
        chunks = text_splitter.create_documents([cleaned_text])
        for chunk in chunks:
            chunk.metadata = {"source": filename}
            processed_docs.append(chunk)

    return processed_docs

### 멀티턴 테스트 (문제 있음)

In [ ]:
import os
import time
from operator import itemgetter
from typing import Dict, Any
from langchain_core.runnables import ConfigurableFieldSpec
from datetime import datetime, timedelta
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, Runnable
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import FileChatMessageHistory
# from langchain.schema.runnable.with_resources import RunnableWithMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from config import Config

class ChatBot:
    def __init__(self, retriever: Any, llm: Any):
        self.retriever = retriever
        self.llm = llm
        self.history_dir = Config.HISTORY_DIR
        self.chain = self._create_chain()

        # 스케줄러를 사용하여 오래된 파일 자동 삭제
        # BackgroundScheduler 등을 사용하면 더 안정적이나, 여기서는 간단한 예시로
        os.makedirs(self.history_dir, exist_ok=True)
        # 매번 인스턴스 생성 시 파일을 정리하도록 구현
        self._clean_old_history_files()

    def _get_session_history(self, session_id: str) -> FileChatMessageHistory:
        """
        세션 ID에 해당하는 FileChatMessageHistory 객체를 반환합니다.
        """
        file_path = os.path.join(self.history_dir, f"{session_id}.json")
        return FileChatMessageHistory(file_path=file_path)

    def _create_chain(self) -> RunnableWithMessageHistory:
        """
        자동으로 대화 기록을 관리하는 챗봇 체인을 생성합니다.
        """
        question_generator_prompt = ChatPromptTemplate.from_messages([
            ("system", "주어진 대화 기록과 새로운 질문을 바탕으로 검색에 사용할 질문을 생성하세요."),
            ("human", "대화 기록:\n{chat_history}\n\n새로운 질문: {question}\n\n검색 질문:")
        ])

        answer_prompt = ChatPromptTemplate.from_messages([
            ("system",
             "다음 문맥만을 근거로 질문에 답변하세요.\n"
             "- 반드시 한국어로 답변하세요.\n"
             "- 문맥에 없는 내용은 '모르겠습니다'라고 답하세요.\n"
             "- 금액/기한/요건은 원문과 일치하게 유지하세요."
            ),
            ("human", "문맥:\n{context}\n\n질문:\n{question}")
        ])

        question_generator_chain = question_generator_prompt | self.llm | StrOutputParser()

        document_chain = (
            {"context": itemgetter("context"), "question": itemgetter("question")}
            | answer_prompt | self.llm | StrOutputParser()
        )

        chain_without_history = (
            RunnablePassthrough.assign(
                # question_generator_chain은 chat_history와 question을 모두 받음
                query=question_generator_chain
            ) |
            {
                "context": itemgetter("query") | self.retriever,
                "question": itemgetter("question")
            } |
            document_chain
        )

        chain_with_history = RunnableWithMessageHistory(
            chain_without_history,
            self._get_session_history,
            input_messages_key="question",
            history_messages_key="chat_history"
        )

        return chain_with_history

    def _clean_old_history_files(self, days_to_keep: int = 10):
        """
        10일보다 오래된 채팅 기록 파일을 삭제합니다.
        """
        now = time.time()
        for filename in os.listdir(self.history_dir):
            file_path = os.path.join(self.history_dir, filename)
            if os.path.isfile(file_path):
                file_mtime = os.path.getmtime(file_path)
                if file_mtime < now - (days_to_keep * 24 * 60 * 60):
                    print(f"Deleting old file: {file_path}")
                    os.remove(file_path)

    def ask(self, chat_id: str, question: str) -> str:
        """
        특정 채팅 세션 ID에 질문하고 답변을 받습니다.
        """
        config = {"configurable": {"session_id": chat_id}}
        inputs = {"question": question}
        outputs = self.chain.invoke(inputs, config=config)
        return outputs

In [ ]:
    llm = ChatOpenAI(model="gpt-5-nano", temperature=0.2, max_tokens=1000)
    embeddings = OpenAIEmbeddings(model=Config.EMBEDDING_MODEL,openai_api_key=Config.OPENAI_API_KEY)
    vectorstore = FAISS.load_local(Config.VECTOR_DB_PATH, embeddings, allow_dangerous_deserialization=True)
    chatbot = ChatBot(llm, vectorstore.as_retriever())

    id = args.id if args.id is not None else str(uuid.uuid4())
    print('현재 chat id: ', id)
    response = chatbot.ask(id, args.query)
    print("\n[최종 답변]")
    print(f"\n{response}")